In [1]:
import os
import re 
import pandas as pd
import numpy as np

from typing import List, Union, Tuple
from tqdm.auto import tqdm
tqdm.pandas()


In [2]:
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
from pgvector.sqlalchemy import Vector
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine, insert, select, text, Integer, String, Text
from sqlalchemy.orm import declarative_base, mapped_column, Session


In [4]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
hostname = os.environ["HOSTNAME"]
#database = os.environ["DATABASE"]
database = "postgres"  # os.environ["DATABASE"]
port = os.environ["PORT"]

db_credentials = f"postgresql://{username}:{password}@{hostname}:{port}/{database}"


In [6]:
engine = create_engine(db_credentials)
with engine.connect() as conn:
    conn.execute(text('CREATE EXTENSION IF NOT EXISTS vector'))
    conn.commit()

Base = declarative_base()

In [7]:
df = pd.read_csv("ctgov_20230608_instruct_embedding.csv", index_col="nct_id")
df

,brief_title,official_title,baseline_measurements,brief_summaries,detailed_descriptions,criteria,gender,minimum_age,maximum_age,facilities,city,state,zip,country,recruitment_details,pre_assignment_details,study_type,embeddings
nct_id,,,,,,,,,,,,,,,,,,
NCT00000143,Studies of Ocular Complications of AIDS (SOCA)...,Studies of Ocular Complications of AIDS (SOCA)...,NaN,"To compare the newest CMV retinitis drug, cido...",Cytomegalovirus (CMV) is among the most freque...,Inclusion criteria:\n\nAge 13 years or older\n...,All,13 Years,NaN,"University of South Florida, MDC Box 21",Tampa,Texas,94143,United States,June 1997,NaN,Interventional,[ 1.26221206e-03 -2.09183618e-02 -5.64291142e-...
NCT00000378,Antidepressant Treatment of Melancholia in Lat...,Antidepressant Treatment of Melancholia in Lat...,NaN,The purpose of this study is to compare the sa...,To compare the efficacy and safety of a select...,Inclusion Criteria:\n\n-\n\nPatients must have...,All,60 Years,95 Years,1051 Riverside Drive,New York,New York,10032,United States,NaN,NaN,Interventional,[-4.05032970e-02 -1.93984844e-02 -6.59292657e-...
NCT00000620,Action to Control Cardiovascular Risk in Diabe...,Action to Control Cardiovascular Risk in Diabe...,NaN,The purpose of this study is to prevent major ...,"BACKGROUND:\n\nCurrently, about 17 million Ame...",Inclusion Criteria:\n\nDiagnosed with type 2 d...,All,40 Years,79 Years,Wake Forest University,Winston-Salem,Washington,98195,United States,All participants had established type 2 diabet...,Eligible participants provided evidence of abi...,Interventional,[-2.31428873e-02 1.43336959e-03 2.46392819e-...
NCT00001151,"Studies With 1,25-Dihydroxycholecalciferol","Studies With 1,25-Dihydroxycholecalciferol",NaN,Vitamin D in the diet undergoes changes in the...,"Patients with extreme resistance to 1,25-dihyd...",INCLUSION CRITERIA:\n\nPatients with hereditar...,All,NaN,NaN,"National Institutes of Health Clinical Center,...",Bethesda,Maryland,20892,United States,NaN,NaN,Interventional,[-4.01057079e-02 6.19083084e-03 1.18229026e-...
NCT00001213,Cysteamine Eye Drops to Treat Corneal Crystals...,Trial of Topical Cysteamine in the Treatment o...,Although 328 participants were initially enrol...,Cystinosis is an inherited disease that result...,"Protocol 86-EI-0062 began as a randomized, dou...",INCLUSION CRITERIA:\n\nPatients must have a do...,All,2 Years,NaN,"National Institutes of Health Clinical Center,...",Bethesda,Maryland,20892,United States,NaN,NaN,Interventional,[-8.30913661e-04 -2.10672207e-02 -1.06763479e-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCT05490771,Testing Copanlisib as a Potential Targeted Tre...,Phase II Study of Copanlisib in Patients With ...,NaN,This phase II MATCH treatment trial identifies...,PRIMARY OBJECTIVE:\n\nI. To evaluate the propo...,Inclusion Criteria:\n\nPatients must have met ...,All,18 Years,NaN,ECOG-ACRIN Cancer Research Group,Philadelphia,Pennsylvania,19103,United States,"Subprotocol Z1F was activated on June 20, 2018...",The PIK3CA mutations status was determined by ...,Interventional,[-1.56701133e-02 -7.25318911e-03 1.66935958e-...
NCT05502081,Clinical Study to Compare Efficacy and Safety ...,Clinical Study to Evaluate the Possible Effica...,0. Uninfected\n\nAmbulatory mild disease\n\nAs...,Introduction:\n\nCorona Virus induced disease ...,I. INTRODUCTION\n\n1.1. COVID-19 overview and ...,Inclusion Criteria:\n\nage more than 12 years ...,All,12 Years,NaN,El-gomhoria St,Mansoura,El-dkhalia,050,Egypt,from 1/11/2021 to 29/5/2022 at isolation hospi...,assignment is applied after admission of parti...,Interventional,[-0.01704628 -0.02524983 0.01437382 0.009319...
NCT05594173,Chewing and Oral Processing of Solid Food,Chewing and Oral Processing of Solid Food in H...,NaN,Food texture modification is commonly used as ...,Aim: To explore chewing and oral processing be...,Inclusion Criteria:\n\nHealthy adults under ag...,All,18 Years,60 Years,Toronto Rehabilitation I

In [8]:
# embeddings = df["embeddings"].apply(lambda x: np.fromstring(re.sub(r'[\n\[\]]', '', x), sep=' ').astype(np.float32))
embeddings = df["embeddings"].copy()
df = df.drop(columns=["embeddings"], axis=1)
df

,brief_title,official_title,baseline_measurements,brief_summaries,detailed_descriptions,criteria,gender,minimum_age,maximum_age,facilities,city,state,zip,country,recruitment_details,pre_assignment_details,study_type
nct_id,,,,,,,,,,,,,,,,,
NCT00000143,Studies of Ocular Complications of AIDS (SOCA)...,Studies of Ocular Complications of AIDS (SOCA)...,NaN,"To compare the newest CMV retinitis drug, cido...",Cytomegalovirus (CMV) is among the most freque...,Inclusion criteria:\n\nAge 13 years or older\n...,All,13 Years,NaN,"University of South Florida, MDC Box 21",Tampa,Texas,94143,United States,June 1997,NaN,Interventional
NCT00000378,Antidepressant Treatment of Melancholia in Lat...,Antidepressant Treatment of Melancholia in Lat...,NaN,The purpose of this study is to compare the sa...,To compare the efficacy and safety of a select...,Inclusion Criteria:\n\n-\n\nPatients must have...,All,60 Years,95 Years,1051 Riverside Drive,New York,New York,10032,United States,NaN,NaN,Interventional
NCT00000620,Action to Control Cardiovascular Risk in Diabe...,Action to Control Cardiovascular Risk in Diabe...,NaN,The purpose of this study is to prevent major ...,"BACKGROUND:\n\nCurrently, about 17 million Ame...",Inclusion Criteria:\n\nDiagnosed with type 2 d...,All,40 Years,79 Years,Wake Forest University,Winston-Salem,Washington,98195,United States,All participants had established type 2 diabet...,Eligible participants provided evidence of abi...,Interventional
NCT00001151,"Studies With 1,25-Dihydroxycholecalciferol","Studies With 1,25-Dihydroxycholecalciferol",NaN,Vitamin D in the diet undergoes changes in the...,"Patients with extreme resistance to 1,25-dihyd...",INCLUSION CRITERIA:\n\nPatients with hereditar...,All,NaN,NaN,"National Institutes of Health Clinical Center,...",Bethesda,Maryland,20892,United States,NaN,NaN,Interventional
NCT00001213,Cysteamine Eye Drops to Treat Corneal Crystals...,Trial of Topical Cysteamine in the Treatment o...,Although 328 participants were initially enrol...,Cystinosis is an inherited disease that result...,"Protocol 86-EI-0062 began as a randomized, dou...",INCLUSION CRITERIA:\n\nPatients must have a do...,All,2 Years,NaN,"National Institutes of Health Clinical Center,...",Bethesda,Maryland,20892,United States,NaN,NaN,Interventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCT05490771,Testing Copanlisib as a Potential Targeted Tre...,Phase II Study of Copanlisib in Patients With ...,NaN,This phase II MATCH treatment trial identifies...,PRIMARY OBJECTIVE:\n\nI. To evaluate the propo...,Inclusion Criteria:\n\nPatients must have met ...,All,18 Years,NaN,ECOG-ACRIN Cancer Research Group,Philadelphia,Pennsylvania,19103,United States,"Subprotocol Z1F was activated on June 20, 2018...",The PIK3CA mutations status was determined by ...,Interventional
NCT05502081,Clinical Study to Compare Efficacy and Safety ...,Clinical Study to Evaluate the Possible Effica...,0. Uninfected\n\nAmbulatory mild disease\n\nAs...,Introduction:\n\nCorona Virus induced disease ...,I. INTRODUCTION\n\n1.1. COVID-19 overview and ...,Inclusion Criteria:\n\nage more than 12 years ...,All,12 Years,NaN,El-gomhoria St,Mansoura,El-dkhalia,050,Egypt,from 1/11/2021 to 29/5/2022 at isolation hospi...,assignment is applied after admission of parti...,Interventional
NCT05594173,Chewing and Oral Processing of Solid Food,Chewing and Oral Processing of Solid Food in H...,NaN,Food texture modification is commonly used as ...,Aim: To explore chewing and oral processing be...,Inclusion Criteria:\n\nHealthy adults under ag...,All,18 Years,60 Years,Toronto Rehabilitation Institute - University ...,Toronto,Ontario,M5G 2A2,Canada,NaN,NaN,Observational


In [9]:
embeddings = embeddings.progress_apply(lambda x: np.fromstring(re.sub(r'[\n\[\]]', '', x), sep=' ').astype(np.float32).tolist())


  0%|          | 0/34983 [00:00<?, ?it/s]

In [10]:
# embeddings[0]

In [11]:
display(df.columns)
display(df.index[0])

Index(['brief_title', 'official_title', 'baseline_measurements',
       'brief_summaries', 'detailed_descriptions', 'criteria', 'gender',
       'minimum_age', 'maximum_age', 'facilities', 'city', 'state', 'zip',
       'country', 'recruitment_details', 'pre_assignment_details',
       'study_type'],
      dtype='object')

'NCT00000143'

In [12]:
class Document(Base):
    __tablename__ = 'instruct'
    
    nct_id = mapped_column(String, primary_key=True)
    brief_title = mapped_column(Text)
    official_title = mapped_column(Text)
    baseline_measurements = mapped_column(Text)
    brief_summaries = mapped_column(Text)
    detailed_descriptions = mapped_column(Text)
    criteria = mapped_column(Text)
    gender = mapped_column(Text)
    minimum_age = mapped_column(Text)
    maximum_age = mapped_column(Text)
    facilities = mapped_column(Text)
    city = mapped_column(Text)
    state = mapped_column(Text)
    zip = mapped_column(Text)
    country = mapped_column(Text)
    recruitment_details = mapped_column(Text)
    pre_assignment_details = mapped_column(Text)
    study_type = mapped_column(Text)
    embedding = mapped_column(Vector(768))


Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)


In [13]:
# df = df.fillna("None")

In [14]:
documents = df.to_dict(orient="records")

In [15]:
documents = [
    dict(
        nct_id=df.index[i], 
        brief_title=df["brief_title"].iloc[i], 
        official_title=df["official_title"].iloc[i], 
        baseline_measurements=df["baseline_measurements"].iloc[i], 
        brief_summaries=df["brief_summaries"].iloc[i], 
        detailed_descriptions=df["detailed_descriptions"].iloc[i], 
        criteria=df["criteria"].iloc[i], 
        gender=df["gender"].iloc[i], 
        minimum_age=df["minimum_age"].iloc[i], 
        maximum_age=df["maximum_age"].iloc[i], 
        facilities=df["facilities"].iloc[i], 
        city=df["city"].iloc[i], 
        state=df["state"].iloc[i], 
        zip=df["zip"].iloc[i], 
        country=df["country"].iloc[i], 
        recruitment_details=df["recruitment_details"].iloc[i], 
        pre_assignment_details=df["pre_assignment_details"].iloc[i], 
        study_type=df["study_type"].iloc[i], 
        embedding=embedding
    ) for i, embedding in enumerate(embeddings)
]


In [22]:
# documents[0]

In [17]:
session = Session(engine)
session.execute(insert(Document), documents)


In [18]:
session.commit()


In [19]:
doc = session.get(Document, "NCT00000143")
neighbors = session.scalars(
    select(Document)
    .filter(Document.nct_id != doc.nct_id)
    .order_by(Document.embedding.cosine_distance(doc.embedding))
    .limit(5))
for neighbor in neighbors:
    print(neighbor.nct_id)

NCT04492475
NCT02541409
NCT04401579
NCT00084149
NCT04640168
NCT02387957
